In [1]:
import os
import json
import pdfplumber
from openai import OpenAI  # OpenAI APIクライアント（現行コードに準拠）
from pyvis.network import Network

In [ ]:
# ======= 設定（必要に応じて変更可能） =======
PDF_FILE_PATH = r"DocumentSource\JP7601283B1.pdf"  # 読み取るPDFファイルのパス
PAGE_START = 6    # 読み取り開始ページ（1-based）
PAGE_END = 8      # 読み取り終了ページ（1-based, 終了ページも含む）
CHUNK_SIZE = 400  # テキストチャンクのサイズ
CHUNK_OVERLAP = 100  # チャンク間の重複部分
OPENAI_API_KEY = 'sk-proj-xxxxxxxxxxx'  # OpenAI APIキー
OUTPUT_KNOWLEDGE_DB_FILE = "knowledge_db.json"  # ナレッジDBの保存先
OUTPUT_GRAPH_HTML = "graph.html"  # グラフ可視化結果のHTML出力ファイル
DOC_ID = "doc_1"  # ドキュメントID（任意の識別子）
# =============================================

# OpenAI APIクライアントの初期化
client = OpenAI(api_key=OPENAI_API_KEY)

# 技術知識抽出用のプロンプトテンプレート
PROMPT_TEMPLATE = (
    "以下のテキストから、技術的な手法、プロセス、パラメータ、用語、及びそれらの関係性を抽出し、JSON形式で返してください。"
    "なお用語については頻出度の高い上位5つの用語を抽出してください。テキスト:\n\n{chunk}"
)


In [4]:
# --- PDFからテキストを抽出 ---
def extract_text_from_pdf(pdf_path, start_page, end_page):
    """
    指定したPDFファイルから、start_page～end_page（1-based）のテキストを抽出して連結した文字列を返す
    """
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        # 1-basedのページ番号を0-indexに変換して抽出
        for page in pdf.pages[start_page - 1:end_page]:
            texts.append(page.extract_text())
    return "\n".join(texts)

# --- テキストをチャンク分割 ---
def split_text(text, chunk_size, chunk_overlap):
    """
    テキストを指定サイズのチャンクに分割する（チャンク間は指定分重複）
    """
    chunks = []
    start = 0
    text_length = len(text)
    while start < text_length:
        end = start + chunk_size
        chunks.append(text[start:end])
        start += (chunk_size - chunk_overlap)
    return chunks

# --- OpenAI API呼び出し ---
def call_openai_api(chunks):
    """
    各チャンクに対してOpenAI APIを呼び出し、抽出した技術知識のレスポンスをリストで返す
    """
    responses = []
    for i, chunk in enumerate(chunks):
        prompt = PROMPT_TEMPLATE.format(chunk=chunk)
        print(f"Processing chunk {i+1}:\n{chunk}\n")
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "あなたは技術知識抽出の専門家です。"},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000,
                temperature=0.2
            )
            extracted = response.choices[0].message.content.strip()
            print("Response:")
            print(extracted)
            responses.append(extracted)
        except Exception as e:
            print(f"Error processing chunk {i+1}: {e}")
    return responses

# --- レスポンスのパース ---
def parse_response(resp_str):
    """
    APIレスポンスからマークダウン部分（```json ... ```）を除去し、JSONオブジェクトとして返す
    """
    lines = resp_str.splitlines()
    if lines and lines[0].startswith("```"):
        lines = lines[1:]
    if lines and lines[-1].startswith("```"):
        lines = lines[:-1]
    json_str = "\n".join(lines)
    return json.loads(json_str)

# --- ナレッジDBの読み込み・初期化 ---
def load_knowledge_db(db_file):
    """
    ナレッジDBのJSONファイルを読み込む。存在しない場合は初期構造を返す。
    """
    if os.path.exists(db_file):
        with open(db_file, "r", encoding="utf-8") as f:
            return json.load(f)
    else:
        return {"documents": [], "graph": {"nodes": [], "edges": []}}

def save_knowledge_db(db, db_file):
    """
    ナレッジDBをJSONファイルに保存する
    """
    with open(db_file, "w", encoding="utf-8") as f:
        json.dump(db, f, ensure_ascii=False, indent=2)

# --- ナレッジDBの更新 ---
def update_knowledge_db(db, extracted_knowledge, full_text, doc_id):
    """
    抽出した技術知識と全文テキストをドキュメントエントリとしてDBに追加し、
    各知識項目をグラフのノード・エッジとして更新する
    """
    # ドキュメントエントリの追加
    doc_entry = {
        "id": doc_id,
        "full_text": full_text,
        "extracted_knowledge": extracted_knowledge
    }
    db["documents"].append(doc_entry)
    
    # ドキュメントノードを追加（重複チェック）
    if not any(node["id"] == doc_id for node in db["graph"]["nodes"]):
        db["graph"]["nodes"].append({
            "id": doc_id,
            "label": doc_id,
            "group": "document"
        })
    
    # カテゴリごとにノード・エッジを追加
    categories = {
        "技術的な手法": "technique",
        "プロセス": "process",
        "パラメータ": "parameter",
        "用語": "term"
    }
    for key, group in categories.items():
        items = extracted_knowledge.get(key, [])
        if isinstance(items, dict):
            items = list(items.keys())
        for item in items:
            # ノード追加（重複チェック）
            if not any(node["id"] == item for node in db["graph"]["nodes"]):
                db["graph"]["nodes"].append({
                    "id": item,
                    "label": item,
                    "group": group
                })
            # ドキュメントと知識項目間のエッジを追加
            db["graph"]["edges"].append({
                "source": doc_id,
                "target": item,
                "label": key
            })
    
    # 「関係性」の処理
    relationships = extracted_knowledge.get("関係性", {})
    if isinstance(relationships, dict):
        for src, targets in relationships.items():
            # ソースノード追加
            if not any(node["id"] == src for node in db["graph"]["nodes"]):
                db["graph"]["nodes"].append({
                    "id": src,
                    "label": src,
                    "group": "relation"
                })
            for target in targets:
                if not any(node["id"] == target for node in db["graph"]["nodes"]):
                    db["graph"]["nodes"].append({
                        "id": target,
                        "label": target,
                        "group": "relation"
                    })
                db["graph"]["edges"].append({
                    "source": src,
                    "target": target,
                    "label": "関連"
                })
    return db

# --- グラフの可視化 ---
def visualize_graph(db, output_filename):
    """
    ナレッジDB内のグラフ情報をpyvisで可視化し、HTMLファイルとして出力する
    """
    net = Network(height="600px", width="100%", directed=False, notebook=False)
    # ノード追加
    for node in db["graph"]["nodes"]:
        net.add_node(node["id"], label=node["label"], group=node["group"])
    # エッジ追加
    for edge in db["graph"]["edges"]:
        net.add_edge(edge["source"], edge["target"], label=edge["label"])
    # HTML生成とファイル保存
    html = net.generate_html()
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(html)
    print(f"グラフが '{output_filename}' として保存されました。")



In [6]:
# --- メイン処理 ---
def main():
    # 1. PDFからテキストを抽出
    full_text = extract_text_from_pdf(PDF_FILE_PATH, PAGE_START, PAGE_END)
    print("PDFからテキストを抽出しました。")
    
    # 2. テキストをチャンクに分割
    chunks = split_text(full_text, CHUNK_SIZE, CHUNK_OVERLAP)
    print(f"テキストを {len(chunks)} 個のチャンクに分割しました。")
    
    # 3. 各チャンクをOpenAI APIに渡して技術知識を抽出
    api_responses = call_openai_api(chunks)
    print("APIからの応答を受信しました。")
    
    # 4. 各レスポンスをパースして統合（単純なマージ処理）
    extracted_knowledge_all = {}
    for response in api_responses:
        try:
            parsed = parse_response(response)
            for key, value in parsed.items():
                if key in extracted_knowledge_all:
                    if isinstance(extracted_knowledge_all[key], list) and isinstance(value, list):
                        extracted_knowledge_all[key].extend(value)
                    elif isinstance(extracted_knowledge_all[key], dict) and isinstance(value, dict):
                        extracted_knowledge_all[key].update(value)
                else:
                    extracted_knowledge_all[key] = value
        except Exception as e:
            print(f"レスポンスのパース中にエラーが発生しました: {e}")
    print("抽出された技術知識（統合結果）:")
    print(json.dumps(extracted_knowledge_all, ensure_ascii=False, indent=2))
    
    # 5. ナレッジDBの読み込みまたは初期化
    db = load_knowledge_db(OUTPUT_KNOWLEDGE_DB_FILE)
    
    # 6. ナレッジDBを更新（ドキュメントエントリとグラフ情報の追加）
    db = update_knowledge_db(db, extracted_knowledge_all, full_text, DOC_ID)
    
    # 7. 更新したナレッジDBを保存
    save_knowledge_db(db, OUTPUT_KNOWLEDGE_DB_FILE)
    print(f"ナレッジDBが '{OUTPUT_KNOWLEDGE_DB_FILE}' に保存されました。")
    
    # 8. グラフを可視化しHTMLとして出力
    visualize_graph(db, OUTPUT_GRAPH_HTML)

if __name__ == "__main__":
    main()


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

PDFからテキストを抽出しました。
テキストを 16 個のチャンクに分割しました。
Processing chunk 1:
(6) JP 7601283 B1 2024.12.17
膜分離方法の一例である逆浸透法によれば、塩分等の溶質を含んだ被処理液体を、該液
体の浸透圧以上の圧力をもって逆浸透膜に透過させることで、溶質分が低減された処理液
体と濃縮液体とに分離することができる。この技術は、例えば海水、かん水、有害物を含
んだ水から飲料水レベルの水を得ることも可能であり、また、工業用超純水の製造、排液
体処理、有価物の回収などにも用いられている。
【０００４】
分離膜の種類、被処理液体の水質、運転方法などに依って程度こそ異なるものの、分離
膜は経時的に膜性能が低下する。そのため、液体処理装置の設計や、安定操業の実現には
、分離膜の分離性能の経時変化を可能な限り正確に知ることが望ましい。
【０００５】 10
分離膜の分離性能の経時変化が生じる原因は、洗浄に伴う化学的な損傷、無機塩のスケ
ール蓄積、微生物由来のファ

Response:
```json
{
  "技術的手法": ["逆浸透法", "膜分離方法"],
  "プロセス": ["被処理液体の浸透圧以上の圧力をもって逆浸透膜に透過させること"],
  "パラメータ": ["分離膜の種類", "被処理液体の水質", "運転方法", "分離膜の分離性能の経時変化"],
  "用語": {
    "1": "逆浸透法",
    "2": "分離膜",
    "3": "膜性能",
    "4": "液体処理装置",
    "5": "安定操業"
  },
  "関係性": {
    "逆浸透法": ["塩分等の溶質を含んだ被処理液体を透過させる", "溶質分が低減された処理液体と濃縮液体に分離する"],
    "分離膜": ["経時的に膜性能が低下する", "分離性能の経時変化を正確に知ることが重要"]
  }
}
```
Processing chunk 2:
には
、分離膜の分離性能の経時変化を可能な限り正確に知ることが望ましい。
【０００５】 10
分離膜の分離性能の経時変化が生じる原因は、洗浄に伴う化学的な損傷、無機塩のスケ
ール蓄積、微生物由来のファウリングなど様々あり、その中の一つとして、被